[View in Colaboratory](https://colab.research.google.com/github/marksandler2/models/blob/master/research/slim/nets/mobilenet/mobilenet_example.ipynb)

>[Prerequisites (downloading tensorflow_models and checkpoints)](#scrollTo=T_cETKXHDTXu)

>[Checkpoint based inference](#scrollTo=fxMe7_pkk_Vo)

>[Frozen inference](#scrollTo=PlwvpK3ElBk6)



# Prerequisites (downloading tensorflow_models and checkpoints)

In [ ]:
!git clone https://github.com/tensorflow/models

In [12]:
from __future__ import print_function
from IPython import display 
base_name = 'resnet_v1_101_2016_08_28.tar.gz' #@param
url = 'http://download.tensorflow.org/models/resnet_v1_101_2016_08_28.tar.gz'
print('Downloading from ', url)
#!wget {url}
print('Unpacking')
!tar -xvf {base_name}.tgz
checkpoint = 'resnet_v1_101.ckpt'

display.clear_output()
print('Successfully downloaded checkpoint from ', url,
      '. It is available as', checkpoint)


Successfully downloaded checkpoint from  http://download.tensorflow.org/models/resnet_v1_101_2016_08_28.tar.gz . It is available as resnet_v1_101.ckpt


In [43]:
!wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O panda.jpg

--2018-04-05 12:40:08--  https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG
Resolving upload.wikimedia.org (upload.wikimedia.org)... 198.35.26.112, 2620:0:863:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|198.35.26.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg’

panda.jpg           100%[===================>] 113.35K  --.-KB/s    in 0.01s   

2018-04-05 12:40:08 (7.67 MB/s) - ‘panda.jpg’ saved [116068/116068]



In [1]:
# setup path
import sys
sys.path.append('/home/krishnashankar/models/research/slim')

# Checkpoint based inference

In [2]:
# import tensorflow as tf
from nets.mobilenet import mobilenet_v2
from nets import resnet_v1
slim = tf.contrib.slim
tf.reset_default_graph()

# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.Print(tf.expand_dims(image, 0), [tf.shape(image)])

images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (224, 224))


                                          

NameError: name 'tf' is not defined

In [18]:
import tensorflow as tf
from nets.mobilenet import mobilenet_v2
from nets import resnet_v1
tf.reset_default_graph()
slim = tf.contrib.slim
# For simplicity we just decode jpeg inside tensorflow.
# But one can provide any input obviously.
file_input = tf.placeholder(tf.string, ())

image = tf.image.decode_jpeg(tf.read_file(file_input))

images = tf.Print(tf.expand_dims(image, 0), [tf.shape(image)[1:3]])

images = tf.cast(images, tf.float32) / 128.  - 1
images.set_shape((None, None, None, 3))
images = tf.image.resize_images(images, (532, 532), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
shapey = tf.Print(tf.shape(images), [tf.shape(images)[1:3]])
# Note: arg_scope is optional for inference.

# Restore using exponential moving average since it produces (1.5-2%) higher 
# accuracy
with tf.variable_scope("feature_extractor"):
    with tf.contrib.slim.arg_scope(resnet_v1.resnet_arg_scope()):
        logits,endpoints = resnet_v1.resnet_v1_101(images,
                                              is_training=False,
                                              reuse=None,
                                              output_stride=16,
                                              global_pool=False)

    with tf.name_scope("concat_blocks"):
        resnet_block_1 = endpoints['feature_extractor/resnet_v1_101/block1']
        resnet_block_2 = endpoints['feature_extractor/resnet_v1_101/block2']
        resnet_block_3 = endpoints['feature_extractor/resnet_v1_101/block3']                                                    
        resized_block_2 = tf.image.resize_images(resnet_block_2, tf.shape(resnet_block_1)[1:3],align_corners=True)
        resized_block_3 = tf.image.resize_images(resnet_block_3, tf.shape(resnet_block_1)[1:3],align_corners=True)
    
        concatenated_blocks = tf.concat([resnet_block_1, resized_block_2, resized_block_3], 3)
        concat_print = tf.Print(concatenated_blocks, [tf.shape(concatenated_blocks)[1:3]])
    
    with slim.arg_scope(
            [slim.conv2d],
            weights_regularizer=slim.l2_regularizer(0.001),
            weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
            reuse=None):
    
        net = slim.conv2d(concat_print,
                        3,
                        kernel_size=1,
                        activation_fn=None)
    with tf.name_scope("transpose"):
        net2 = tf.identity(net, name='the_output')
        net3 = slim.conv2d_transpose(net2,3,4, 2)
        net4 = slim.conv2d_transpose(net3,3,4, 2)
        net5 = slim.conv2d_transpose(net4,3,4, 2)
        shapex = tf.Print(tf.shape(net5), [tf.shape(net5)])
#ema = tf.train.ExponentialMovingAverage(0.999)
#vars = ema.variables_to_restore()
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()  


In [9]:
from IPython.display import clear_output, Image, HTML
from IPython.display import display as dips
import numpy as np
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    dips(HTML(iframe))
    

In [19]:
from IPython import display
import pylab
from datasets import imagenet
import PIL
from PIL import Image



with tf.Session() as sess:
  tf.train.write_graph(sess.graph_def, '/tmp/', 'train.pbtxt')

  #saver.restore(sess,  checkpoint)
  sess.run(init_op)
  x = shapex.eval(feed_dict={file_input: 'panda.jpg'})
#label_map = imagenet.create_readable_names_for_imagenet_labels()  
#print("Top 1 prediction: ", x.argmax(),label_map[x.argmax()], x.max())
print(x)
#display.display(display.Image('panda.jpg'))

#img = Image.fromarray(x, 'RGB')
#img.show()

[  1 536 536   3]


# Frozen inference

In [7]:
import numpy as np
img = np.array(PIL.Image.open('panda.jpg').resize((224, 224))).astype(np.float) / 128 - 1
gd = tf.GraphDef.FromString(open(base_name + '_frozen.pb', 'rb').read())
inp, predictions = tf.import_graph_def(gd,  return_elements = ['input:0', 'MobilenetV2/Predictions/Reshape_1:0'])

In [8]:
with tf.Session(graph=inp.graph):
  x = predictions.eval(feed_dict={inp: img.reshape(1, 224,224, 3)})

label_map = imagenet.create_readable_names_for_imagenet_labels()  
print("Top 1 Prediction: ", x.argmax(),label_map[x.argmax()], x.max())

Top 1 Prediction:  389 giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca 0.8914295
